### Optimisation du code en cours

#### Mount the Google Drive in order to access to the files which are located on our drive


In [1]:
from google.colab import drive
drive._mount('/content/drive')

#### Install and import every packages that will be necessary for the analysis

In [2]:
!pip install pickle-mixin
!pip install pyarrow
!pip install pathlib


import pyarrow
import pandas as pd
import numpy as np
import pathlib
from datetime import datetime
import sys
import bz2
import json
import pickle
import seaborn as sns
import matplotlib.pyplot as plt

### Imports, et import du model qu'on utilise ( pour l'instant all-MiniLM-L6-v2 )

In [1]:
#si besoin
#pip install -U sentence-transformers
#pip install gensim

In [3]:
#NLP libraries
import nltk
from nltk import word_tokenize
#from gensim.models import FastText #pas utilisé finalement

In [4]:
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import euclidean_distances
from sentence_transformers import SentenceTransformer

#model = SentenceTransformer(r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\pretrained_model') #pour si la ligne d'en dessous ne marche pas, telecharger le model en local et mettre le path
model = SentenceTransformer('all-MiniLM-L6-v2')


# j'ai testé plysieurs models:

# all-MiniLM-L6-v2: ça passe, treshold à 0.6 ?
#'bert-base-nli-mean-tokens' : deprecated
#all-mpnet-base-v2: ne se télécharge pas
#multi-qa-mpnet-base-dot-v1: ne va pas, tous les scores sont trés haut meme pour 'truc contraires
#multi-qa-distilbert-cos-v1: mouai


### Comparaison des citations (morceau par morceau) aux expressions du paper, et création d'un dataframe avec qids, quote, sim_score_expression1, sim_score_expression2, ..., sim_score_expressionN.

Je decoupe les citations. Pour chaque expression: je compare chaque morceau de la citation avec l'expression, ça donne un score de similarité pour chaque morceau: on prend le max (ou bien faire un teshold ? à voir).  







#### Basé sur la technique du paper Sentence Similarity Techniques for Short vs Variable Length Text using Word Embeddings


#### Extract a small part of the whole quotes file to test (j'ai pas le vrai fichier pre procéssé, c'est pour ça qu'il y a des [ ] dans les citations, mais ça marchera pareil avec le bon fichier

In [5]:
#Extract a small part of the whole file to test 
path = r'C:\Users\flore\Desktop\EPFL\MA1\ada\PROJET\quotes-processed.json.bz2' #path florette
quotes = pd.DataFrame(columns=('qids','quotation'))
i = 0
with bz2.open(path, 'rb') as s_file:
  for instance in s_file:
    instance = json.loads(instance) 
    #print(instance)
    #print(instance['quotation'])
    i = i + 1
    quotes = quotes.append({'qids': instance['qids'], 'quotation': instance['quotation']}, ignore_index=True)
    #print(i)
    if i == 10:
      break

#### append some quotation for testing the efficacy

In [6]:
quotes = quotes.append({'qids': 'Qtest1', 'quotation': 'I know it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest2', 'quotation': 'I dont know if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest3', 'quotation': 'I am sure it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest4', 'quotation': 'I am not sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest5', 'quotation': 'I dont eat an apple'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest6', 'quotation': 'I pretty sure if it is a fool'}, ignore_index=True)
quotes = quotes.append({'qids': 'Qtest7', 'quotation': 'I was sure if it is a fool'}, ignore_index=True)


#### create a dataframe de test with the expression we want to compare, and their score (remplacer aprés avec le vrai)

In [7]:
#create a dataframe with the ewpression we want to compare, and their score
expressions = pd.DataFrame({'expression': ["I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"], 'expression_score': [2.91, 3.02, 3.34, 3.48, 6.02, 6.45]})
expressions.head(3)

,expression,expression_score
0,I'm not sure,2.91
1,I dont know,3.02
2,I suppose,3.34


### Comparaison

#### Chunker les citations pour découper en un certain nombre de mots (j'ai pas trouvé de méthode optimale, alors je tokenize en mots, puis je join le nombre de mots qu'on veut

In [8]:
#optimisé
#First, we separate the word and symbol using tokenisation. This will allow us to then reform n_grams with the desired number of words/symbol.
quotes['n_grams'] = quotes['quotation'].apply(lambda x: nltk.word_tokenize(x))

#Then reform chunk of n words
n = 4 #number in n_gram
quotes['n_grams'] = quotes['n_grams'].apply(lambda x: (list(" ".join(x[i : i+n-1]) for i in range(len(x) - n + 1))))
quotes

,qids,quotation,n_grams
0,[Q367796],Department of Homeland Security was livid and ...,"[Department of Homeland, of Homeland Security,..."
1,[Q20684375],I met them when they just turned 4 and 7. They...,"[I met them, met them when, them when they, wh..."
2,[Q5268447],The delay will have an impact on Slough but th...,"[The delay will, delay will have, will have an..."
3,[Q4864119],The scheme treats addiction as an illness and ...,"[The scheme treats, scheme treats addiction, t..."
4,[Q816459],These actions will allow households who have a...,"[These actions will, actions will allow, will ..."
5,[Q30164281],1. FM is entitled to go straight to press conf...,"[1 . FM, . FM is, FM is entitled, is entitled ..."
6,[Q56255401],"11 straight weeks of pre-season,","[11 straight weeks, straight weeks of, weeks o..."
7,[Q26923564],2019 was a landmark year for Fiverr as we comp...,"[2019 was a, was a landmark, a landmark year, ..."
8,[Q4749380],7pm is when most hospitals change shifts. That...,"[7pm is when, is when most, when most hospital..."
9,[Q970000],A city is not an accident but the result of co...,"[A city is, city is not, is not an, not an acc..."


#### Compare the citations chunk with the expressions and add the comparison score = cosinus similarity (max score for the different chunk) in a new column

In [10]:
#optimisé

#quotes['cosine_similarity'] = quotes['n_grams'].apply(lambda x: list(max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x)) for expression_embedded in model.encode(expressions.expression)))
#quotes['cosine_similarity'] = quotes['n_grams'].apply(lambda x: list(max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x)).flatten() for expression_embedded in model.encode(expressions.expression)))

quotes['cosine_similarity'] = quotes['n_grams'].apply(lambda x: list(max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x)).flatten()[0] for expression_embedded in model.encode(expressions.expression)))


#test pour direct mettre dans des colonnes séparées mais pas encore trouvé comment faire
#quotes['n_grams'].apply(lambda x: quotes[expressions.expression] <- (max(cosine_similarity([n_gram_embedded], [expression_embedded]) for n_gram_embedded in model.encode(x))) for expression_embedded in model.encode(expressions.expression))

quotes.head(20)



,qids,quotation,n_grams,cosine_similarity
0,[Q367796],Department of Homeland Security was livid and ...,"[Department of Homeland, of Homeland Security,...","[0.3032319, 0.3031956, 0.23602816, 0.22850059,..."
1,[Q20684375],I met them when they just turned 4 and 7. They...,"[I met them, met them when, them when they, wh...","[0.2303027, 0.2367206, 0.27510583, 0.29699588,..."
2,[Q5268447],The delay will have an impact on Slough but th...,"[The delay will, delay will have, will have an...","[0.41248465, 0.34659547, 0.50099874, 0.6188506..."
3,[Q4864119],The scheme treats addiction as an illness and ...,"[The scheme treats, scheme treats addiction, t...","[0.27248415, 0.21348144, 0.23313306, 0.2357903..."
4,[Q816459],These actions will allow households who have a...,"[These actions will, actions will allow, will ...","[0.2834597, 0.28215024, 0.22757119, 0.18103966..."
5,[Q30164281],1. FM is entitled to go straight to press conf...,"[1 . FM, . FM is, FM is entitled, is entitled ...","[0.29281023, 0.25658804, 0.33833924, 0.2641971..."
6,[Q56255401],"11 straight weeks of pre-season,","[11 straight weeks, straight weeks of, weeks o...","[0.1850446, 0.1714744, 0.13752353, 0.18754275,..."
7,[Q26923564],2019 was a landmark year for Fiverr as we comp...,"[2019 was a, was a landmark, a landmark year, ...","[0.2398042, 0.22135988, 0.18565738, 0.25002772..."
8,[Q4749380],7pm is when most hospitals change shifts. That...,"[7pm is when, is when most, when most hospital...","[0.26984522, 0.27745345, 0.3020717, 0.23797627..."
9,[Q970000],A city is not an accident but the result of co...,"[A city is, city is not, is not an, not an acc...","[0.31056875, 0.28648853, 0.2216119, 0.25278366..."


In [123]:
display(quotes_similarity.iloc[10:20])#pour ettre chaque score dans une colonne nomée par le nom de la citation
#for exp in zip(expressions.expression, range(len(expressions.expression))):
#    quotes['exp'] = quotes.cosine_similarity
#quotes.head(20)

#df_max_cos_sim = pd.DataFrame(index=("I'm not sure", "I dont know", "I suppose", "I think", "I'm sure", "I know"))
